# Code to calculate F_ST Matrix for Punic project
This is run on v54.1 data from the HDF5.

Development Area. Will be packed into functions for
calculation and plotting notebooks.

In [1]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
from time import time
import h5py as h5py
import allel ### To Calculate Scikit Allel

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

compute-e-16-236.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28
3.7.4 (default, Sep 11 2019, 11:24:51) 
[GCC 6.2.0]


In [2]:
def create_meta_df(f, path_meta="/n/groups/reich/hringbauer/Data/v49.2.anno.csv"):
    """Create and return Meta Dataframe that matches hdf5 in format"""
    samples = f["samples"][:].astype("str")
    df_h5 = pd.DataFrame({"iid":samples})
    df_meta = pd.read_csv(path_meta)
    print(f"Loaded {len(df_meta)} rows from Metafile")
    df = pd.merge(df_h5, df_meta, on="iid", how="left")
    print(f"Created matching Meta Dataframe for h5: {len(df)}")
    df = df.reset_index(drop=True)
    df["clst"] = df["clst"].fillna("Not Available")
    return df

def set_clst(df, idx_list=[], clst_new="", 
             col_iid="iid", col_clst="clst"):
    """Set New Cluster Label to Dataframe"""
    idx = df[col_iid].isin(idx_list)
    df.loc[idx, col_clst] = clst_new
    print(f"Updated {np.sum(idx)} Individuals {col_clst}: {clst_new}")

def get_cluster_idx(df, clst="", col_clst="clst", 
                    age_range=[], exact=False,
                    include_col="include"):
    """Get idcs of all samples within Cluster
    If age_range, only samples in age range"""
    if len(include_col)>0:
        idcs1 = df[include_col]==True
    else:
        idcs1 = True
        
    if exact:
        idcs = np.where((df[col_clst]==clst) & idcs1)[0] 
    else:
        idcs = np.where((df[col_clst].str.contains(clst)) & idcs1)[0]
    
    ### Do additional Filtering
    if len(age_range)>0:  
        pass
    return idcs

###############################
### Calculate the Allele Counts
def get_ph(f, idcs):
    """Sample pseudohaploid data for hdf5 for
    individuals with indices idcs"""
    ads = f["calldata/AD"][:,idcs,:2]
    ads[ads<0]=0 # Set nmissing data to 0
    cov = np.sum(ads, axis=2) # get the coverage per locus/indiviual
    idx = cov>0  # Where there is some coverage
    p = np.divide(ads[:,:,1], cov, where=idx)
    p[~idx]=1
    p = np.clip(p, a_min=0, a_max=1) # Santity check to deal with numerics
    ac = np.random.binomial(1,p)
    ac[~idx] = -1
    return ac

def get_gt(f, idcs):
    """Get diploid genoytpe counts"""
    gt = f["calldata/GT"][:,idcs,:]
    assert(np.min(gt)>=0)
    gt = np.sum(gt, axis=2) # Count #derived variants
    return gt

def calc_ac_from_ph(ph):
    """Calculate allele allele counts for individuals 
    with indices from hdf5 with allele counts only
    ph: Array of pseudo-haploid [l,n]"""
    c_ref=np.sum(ph==0, axis=1) # Sum the Counts over all Individuals
    c_alt=np.sum(ph==1, axis=1) # Sum the Counts over all Individuals
    
    # Double 0,0 no problem, goes to NaN and is then caught by allel
    return np.column_stack((c_ref, c_alt)) # Return the nx2 Allele Counts

def calc_ac_from_gt(gt):
    """Calculate allele allele counts for individuals 
    with indices from hdf5 with allele counts only
    ph: Array of pseudo-haploid [l,n]"""
    c_ref= 2*np.sum(gt==0, axis=1) + np.sum(gt==1, axis=1)
    c_alt= 2*np.sum(gt==2, axis=1) + np.sum(gt==1, axis=1)
    return np.column_stack((c_ref, c_alt)) # Return the nx2 Allele Counts

def get_ac_from_f(f, idcs, ph=True):
    """Get Allele Counts from HDF, 
    grouped for all indivdiuals in idcs
    ph: Whether to use pseudo-haploid or diploid genotypes"""
    if ph:
        ph = get_ph(f, idcs)
        ac = calc_ac_from_ph(ph=ph)
    else:
        print("Using diploid mode...")
        gt = get_gt(f, idcs)
        ac = calc_ac_from_gt(gt)
    return ac

def calculate_ac_pop(clst, f, df, col="clst", exact=False, 
                     ph=True, include_col="include"):
    """Return allele counts for population.
    exact: whether ther is an exact match"""
    idcs = get_cluster_idx(df, clst=clst, exact=exact,
                           col_clst=col, include_col=include_col)
    ac = get_ac_from_f(f, idcs, ph=ph) 
    return ac

def calculate_ac_pops(pops, f, df, col="clst", ph=True, 
                      exact=False, out=True):
    """Calculate list of allele counts [l,2] for pops
    f: hdf5
    df: metafile matching f
    pops: List of populations to extract ACs for"""
    ### Check whether all pops have matches first:
    idcss = [get_cluster_idx(df=df, clst=pop, exact=exact, col_clst=col)
                              for pop in pops]
    counts = np.array(map(len, idcss))
    if np.min(counts)==0:
        idx = np.where(counts==0)[0]
        raise RuntimeError(f"Pops {pop[idx]} not found!")
    
    acs=[]
    for pop in pops:
        idcs = get_cluster_idx(df=df, clst=pop, exact=exact, col_clst=col)
        if len(idcs)==0:
            raise RuntimeWarning(f"No matching iids for {pop} not found!!")
        if out:
            print(f"Calculating counts pop: {pop}, n={len(idcs)}...")
        ac = get_ac_from_f(f, idcs, ph=ph) 
        acs.append(ac)
    return acs

###########################################
###########################################
### Calculate the actual f statistics

def f3_ac(pt, p1, p2, snps_okay=None, blen=1000):
    """Calculate f3 for Allele Counts (lx2 arrays)
    snps_okay: Which SNPs to actually use. If none use all
    blen: Block Nr for Bootstrap
    """
    #f3 = np.mean((pt-p1)*(pt-p2))
    f3 = allel.average_patterson_f3(pt, p1, p2, blen=blen, normed=False)
    return [f3[0], f3[1], f3[2]]  # f4, se, z

def f4_ac(p1, p2, p3, p4, snps_okay=None, blen=1000):
    """Calculate f4 for Allele Counts (lx2 arrays)
    snps_okay: Which SNPs to actually use (If none use all)
    blen: Block Nr for Bootstrap
    """
    f4 = allel.average_patterson_d(p1, p2, p3, p4, blen=blen)
    return [f4[0], f4[1], f4[2]]  # f4, se, z

def fst_ac(p1, p2, blen=1000):
    """Calculate f3 for Allele Counts (lx2 arrays)
    blen: Block Nr for Bootstrap
    A sim wrapper, so later on different methods can be implemented.
    Return fst, se, z value (based on jackkniving)
    """
    res = allel.average_patterson_fst(p1, p2, blen=blen)
    f4, se = res[0], res[1]
    z = f4 / se # Calculate the z-Value
    return [res[0], res[1], z]  # f4, se, z

# Load the Data

In [ ]:
%%time
path_anno = "/n/groups/reich/hringbauer/Data/v54.1.anno.csv"
path_h5 = "/n/groups/reich/hringbauer/git/hapBLOCK/data/hdf5/1240k_v56.3/XXX.h5"
#path_meta = "/n/groups/reich/hringbauer/Data/"/v49.2_punic_meta.tsv"

f = h5py.File(path_h5, "r")
df = create_meta_df(f, path_meta=path_anno)

### Get only the highest coverage individuals

df1 = df.sort_values(by="n_cov_snp", ascending=False).copy()
idx = df1.duplicated(subset="Master ID", keep='first')
print(f"Found {np.sum(idx)} Duplicate")
iids = df1["iid"][~idx]
idx_unique = df["iid"].isin(iids)

df["include"]= False
df.loc[idx_unique, "include"] = True
print(f"Set {np.sum(idx_unique)} Rows to Include=True")
### Merge in the cluster label"s
#df1 = pd.read_csv(path_meta, sep="\t")
### Only include unique,unrelated samples
#df1 = df1[df1["include"]==1].copy().reset_index(drop=True) 
#df1 = pd.merge(df,df1[["iid", "label_region", "include"]], on="iid", how="left")
#df1.loc[df1["label_region"].isnull(), "label_region"]="not assigned"
#assert(len(df1)==len(df))

##########################################
### Prepare dataframe
### Set custom Labels - for Punic analysis
df_context = pd.read_csv("./output/tables/reference_samples_plot.v49.2.tsv", sep="\t")
print(f"Loaded {len(df_context)} Individuals")

In [4]:
df1 = df.copy()   # Copy for adding edits while keeping the original

### Set the Akzhiv Indivdiuals
idx = df["loc"].isnull()
df[idx] ="not assigned"
idx = df["loc"].str.contains("Akhziv")
df1.loc[idx, "clst"] = "Israel_Phoenician"

### Set the Core Punic Individuals
### Old per site

iids_lil_early = ["I12666", "I12847", "I24678", "I24676", "I24675", "I24556", "I22095"] ### Lilybaeum Early
set_clst(df1, idx_list=iids_lil_early, clst_new="Lilybaeum_Early_Punic")

iids_mot_early = ["I4798", "I4799", "I4800", "I22236"] ### Motya Early # 2 Outliers "I7762", "I22232"
set_clst(df1, idx_list=iids_mot_early, clst_new="Motya_Early_Punic")

iids_thar_early = ["I22115", "I22121", "I22096", "I22118", "I22117"] ### Tharros Early I22122 offspring
set_clst(df1, idx_list=iids_thar_early, clst_new="Tharros_Early_Punic")

### Set Africans (High on PCA)
iids_afr_punic =  ["I18193", "I18189", "I22093"]  # "I22113" high but not high enough
set_clst(df1, idx_list=iids_afr_punic, clst_new="Punic_African")

### African Cline (On PCA cline)
iids_afr_cline = ["I21966", "I21984", "I22094", "I22090", "VIL011", "VIL006", "VIL009", "VIL010", "VIL007"] # but not VIL004
set_clst(df1, idx_list=iids_afr_cline, clst_new="Punic_African_Cline")


iids_tunisia_m = ["I20824", "I20825"]
set_clst(df1, idx_list=iids_tunisia_m, clst_new="Tunisia_Mesolithic")

### Drop Indivdiuals
iids_na = ["I22257", "TAF014", "TAF011", "I13394"]  # The African Outlier in the Phoenicians / 2 Related Taforalt /  1 related Sicily_IA/ 2 relatedI22236
for iid in iids_na:
    idx = df["iid"]==iid
    assert(np.sum(idx)>0)
    df1.loc[idx, "clst"] = "ignore"

Updated 7 Individuals clst: Lilybaeum_Early_Punic
Updated 4 Individuals clst: Motya_Early_Punic
Updated 5 Individuals clst: Tharros_Early_Punic
Updated 3 Individuals clst: Punic_African
Updated 9 Individuals clst: Punic_African_Cline
Updated 2 Individuals clst: Tunisia_Mesolithic


In [65]:
df1[df1["clst"].str.contains("Tunisia_LN")]

,iid,Master ID,loc,lat,lon,age,age_range,region,study,clst,mean_cov,n_cov_snp,avg_cov_snp,include_alt,family,sex,contact,include
14430,I22866,I22866,Dukanet el Ketif,NaN,NaN,6727,"4878-4712 calBCE (5910±30 BP, PSUAMS-9397)",Tunisia,Unpublished / Unmarked as relevant to a study,Tunisia_LN,0.702394,842873,3.079,True,n/a (no relatives detected),F,"Pinhasi, Ron",True
18186,I22852,I22852,Hergla,NaN,NaN,5864,"4035-3804 calBCE (5130±25 BP, PSUAMS-9395)",Tunisia,Unpublished / Unmarked as relevant to a study,Tunisia_LN,0.474592,569510,0.819,True,n/a (no relatives detected),M,"Pinhasi, Ron",True


# Calculate F_ST

### Example goto Area 51

# Full Pipeline of Calculating all F_ST

In [6]:
%%time

pops = ["Lilybaeum_Early_Punic", "Motya_Early_Punic", "Tharros_Early_Punic",
        "Morocco_Iberomaurusian", "Morocco_EN.SG", "Morocco_LN.SG", "Punic_African", "Punic_African_Cline", 
        "Israel_Phoenician", "Israel_MLBA", "Spain_IA", "Italy_Sicily_IA_Polizzello",
        "Italy_Sardinia_EBA", "Greece_BA_Mycenaean", "Italy_Sardinia_BA_Nuragic",
        "CanaryIslands_Guanche.SG", "Tunisia_N", "Tunisia_Mesolithic", "Italy_Sicily_MBA", "Spain_Menorca_LBA",
        "Tunisia_LN", "Egypt_ThirdIntermediatePeriod"]
col = "clst"
ph = True

##########################################
### Calculating Allele Frequencies
ns = [len(get_cluster_idx(df=df1, clst=p, 
                           exact=True, col_clst=col)) for p in pops]
print(ns)
assert(np.min(ns)>1) # To have enough data for F_st

print(f"Calculating AF {len(pops)} Populations")
acs = calculate_ac_pops(pops=pops, f=f, df=df1, col=col, 
                        exact=True, out=True, ph=ph)    
print("Finished pre-processing all freqs. Starting Allele Count calculation!")

##########################################
### Calculate Fst from the list of allele counts for all pairs
res, res_h = [], []  # Vector for the Results
pops1, pops2 = [], [] # Vector for the populations pairs

k = len(pops)
for i1, i2 in it.combinations(np.arange(k),2):
    # Save for the population list
    pops1.append(pops[i1])
    pops2.append(pops[i2])    
    ### Do the fst Calculation
    res_new = fst_ac(acs[i1], acs[i2])
    res.append(res_new)
    
print("\nFinished Fst Matrix Calculation!")

[7, 4, 5, 4, 3, 3, 3, 9, 15, 35, 20, 18, 13, 15, 11, 5, 4, 2, 7, 3, 2, 2]
Calculating AF 22 Populations
Calculating counts pop: Lilybaeum_Early_Punic, n=7...
Calculating counts pop: Motya_Early_Punic, n=4...
Calculating counts pop: Tharros_Early_Punic, n=5...
Calculating counts pop: Morocco_Iberomaurusian, n=4...
Calculating counts pop: Morocco_EN.SG, n=3...
Calculating counts pop: Morocco_LN.SG, n=3...
Calculating counts pop: Punic_African, n=3...
Calculating counts pop: Punic_African_Cline, n=9...
Calculating counts pop: Israel_Phoenician, n=15...
Calculating counts pop: Israel_MLBA, n=35...
Calculating counts pop: Spain_IA, n=20...
Calculating counts pop: Italy_Sicily_IA_Polizzello, n=18...
Calculating counts pop: Italy_Sardinia_EBA, n=13...
Calculating counts pop: Greece_BA_Mycenaean, n=15...
Calculating counts pop: Italy_Sardinia_BA_Nuragic, n=11...
Calculating counts pop: CanaryIslands_Guanche.SG, n=5...
Calculating counts pop: Tunisia_N, n=4...
Calculating counts pop: Tunisia_Me

/n/groups/reich/hringbauer/explore_ntbk/jptvenv37/lib/python3.7/site-packages/allel/stats/admixture.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = (ac[:, 0] * ac[:, 1]) / (an * (an - 1))
/n/groups/reich/hringbauer/explore_ntbk/jptvenv37/lib/python3.7/site-packages/allel/stats/fst.py:802: RuntimeWarning: invalid value encountered in true_divide
  vb = num_bsum / den_bsum



Finished Fst Matrix Calculation!
CPU times: user 6min 53s, sys: 19.9 s, total: 7min 13s
Wall time: 15min 35s


In [7]:
ns=[]
for pop in pops:
    idcs = get_cluster_idx(df=df1, clst=pop, 
                           exact=True, col_clst=col)
    ns.append(len(idcs))
    
res = np.array(res) # For better slicing 
### Make F_ST Data Frame
df_fst = pd.DataFrame({"s1": pops1, "s2": pops2,
               "fst": res[:,0], "se": res[:,1], "z": res[:,2]})

### Save the F_ST-Matrix as well as the populaition:
folder = "./output/tables/fst/v49.2/"  # Where to save: fst-mat-hudson

df_fst.to_csv(folder + 'fst.csv', index=False) # Save the Values
print(f"Saved {len(df_fst)} pw. Comparisons.")

### Do the pop size numbers
df_pops = pd.DataFrame({"pop": pops, "n": ns})
df_pops.to_csv(folder + 'pops.csv', index=False) # Save the Values
print(f"Saved {len(df_pops)} populations.")

Saved 231 pw. Comparisons.
Saved 22 populations.


# Area 51

In [21]:
### Calculate the outgroup f3

%%time
pops = ["CanaryIslands_Guanche.SG", "Israel_Phoenician", 
        "Italy_Sardinia_BA_Nuragic"]

acs = calculate_ac_pops(pops=pops, 
                        f=f, df=df1, col="clst", 
                        exact=False, out=True, ph=True)

### Calculate the outgroup f3
#fst_ac(acs[2], acs[3])

fst_ac(acs[1], acs[2])

['Israel_Phoenician',
 'Israel_MLBA',
 'Spain_IA',
 'Italy_Sicily_IA_Polizzello',
 'Italy_Sardinia_EBA',
 'Greece_BA_Mycenaean',
 'Italy_Sardinia_BA_Nuragic',
 'CanaryIslands_Guanche.SG',
 'Tunisia_N',
 'Italy_Sicily_MBA',
 'Spain_Menorca_LBA',
 'Tunisia_LN']

In [8]:
with h5py.File(path_h5, "r") as f:
    print(list(f))
    print(list(f["calldata"]))
    s = f["samples"][:]

['calldata', 'samples', 'variants']


RuntimeError: Unable to get group info (addr overflow, addr = 1166280, size = 328, eoa = 2048)